># Impoting libs for the preprocess the data an train the model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

># Importing both datasets, the Housig prices and test

In [50]:
X_full = pd.read_csv('Housing_prices.csv',index_col='Id')
X_test_full = pd.read_csv('Housing_prices_test.csv', index_col='Id')

># Removing rows with missing target from full data

In [51]:
X_full.dropna(axis=0, subset= ['SalePrice'], inplace= True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis= 1, inplace= True)

># Separating test and training the dataset

In [52]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size= 0.2, random_state=0)

># Selecting categorical columns with low cardinality values

In [53]:
categorical_cols = [col_name for col_name in X_train_full.columns if X_train_full[col_name].nunique() < 10 and X_train_full[col_name].dtype == 'object']

># Selecting numerical columns

In [54]:
numerical_cols = [col_name for col_name in X_train_full.columns if X_train_full[col_name].dtype in ['int64','float64']]

># Keeping the selected columns only making a copy

In [55]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
X_train.head()

In [58]:
# STEP 1 : Improving the performance
# Preprocessing for numerical data
numerical_transformer = SimpleImputer (strategy= 'constant')

In [59]:
# Preprocessing for the categorical data
categorical_transformer = Pipeline(steps= [
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown= 'ignore'))
])

In [60]:
# Bundle both preprocessing from categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
('numerical', numerical_transformer, numerical_cols),
('categorical', categorical_transformer,categorical_cols)        
        ])

In [61]:
# Defining the model
model = RandomForestRegressor(n_estimators=500, random_state=0)

In [62]:
# Bundle the preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)
])

In [63]:
# Preprocessing of training data and fit model
my_pipeline.fit(X_train, y_train)
# Preprocessing of validation data and get predictions
predictions = my_pipeline.predict(X_valid)

In [65]:
# STEP 2: Generating test predictions
# Preprocessing of test data and fit model
predictions_test = my_pipeline.predict(X_test)


In [66]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': predictions_test})
output.to_csv('predictions.csv', index=False)